In [1]:
import zipline
from zipline.api import order_target_percent, record, symbol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from matplotlib import pyplot as plt
from numpy import record
import pandas as pd
import zipline


def initialize(context):
    context.asset = symbol('AAPL')
    context.i = 0
    context.macd_threshold = 0
    
    
def handle_data(context, data):
    context.i += 1
    if context.i < 365:
        return
    
    prices = data.history(context.asset, 'price', bar_count=365, frequency='1d')
    ema12 = prices.ewm(span = 12, adjust = False).mean()
    ema26 = prices.ewm(span = 26, adjust = False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span = 9, adjust = False).mean()
    diff = macd - signal
    
    
    if diff[-1] > context.macd_threshold:
        order_target_percent(context.asset, 1)
    elif diff[-1] < context.macd_threshold:
        order_target_percent(context.asset, 0)
    
    record(macd = macd[-1], signal = signal[-1])
    
start_date = pd.Timestamp('2022-01-01', tz='utc')
end_date = pd.Timestamp('2022-12-31', tz='utc')
capital_base = 10000
benchmark_symbol = '^GSPC'


result = zipline.run_algorithm(start=start_date, end=end_date, initialize=initialize, capital_base=capital_base, handle_data=handle_data)
result[['macd', 'signal']].plot()
plt.show()

ValueError: no data for bundle 'quantopian-quandl' on or before 2023-04-02 08:10:45.942956+00:00
maybe you need to run: $ zipline ingest -b quantopian-quandl